In [1]:
%matplotlib qt

In [2]:
## Dependencies
from libs.timing_diagram import *
from libs.design_functions import *
import numpy as np
import matplotlib.pyplot as plt
from libs.spherical_earth_geometry_radar import *
from libs.radartools.farField import UniformAperture
from libs.ambiguity_functions import *
from unambiguous_imaging_design_analysis import *
import matplotlib as mpl

ChatGpt: Sure! Here's the specification for loading the analysis results from the saved pickle files:

### LOADER
- Load the analysis results from the saved pickle files using pickle module.
- Load the following individual files from the "analysis_results" folder:
    * results_list.pk
    * x_coordinate.pk
    * y_coordinate.pk
    * undersampling_list.pk
    * rasr_free_swath_list.pk
    * core_snr_list.pk
- Assign the loaded data to the corresponding variable names:
    * results_list
    * x_coordinate
    * y_coordinate
    * undersampling_list
    * rasr_free_swath_list
    * core_snr_list

Note: The file names and variable names must match with the ones used in the SAVER section.
Note1: Run the cells preceding and in Phase 0 before or after loading

In [3]:
import pickle
from IPython.display import Javascript

Javascript("Jupyter.notebook.execute_cell_range(1,11)")  #doesn't work. run clls manually up to phase0

# Load the results_list
with open('analysis_results/results_list.pk', 'rb') as f:
    results_list = pickle.load(f)
# Load the x_coordinate
with open('analysis_results/x_coordinate.pk', 'rb') as f:
    x_coordinate = pickle.load(f)
# Load the y_coordinate
with open('analysis_results/y_coordinate.pk', 'rb') as f:
    y_coordinate = pickle.load(f)
# Load the undersampling_list
with open('analysis_results/undersampling_list.pk', 'rb') as f:
    undersampling_list = pickle.load(f)
# Load the rasr_free_swath_list
with open('analysis_results/rasr_free_swath_list.pk', 'rb') as f:
    rasr_free_swath_list = pickle.load(f)
# Load the core_snr_list
with open('analysis_results/core_snr_list.pk', 'rb') as f:
    core_snr_list = pickle.load(f)

In [4]:
import json

with open("analysis_results/settings.json", "r") as read_file:
    data = json.load(read_file)

La = data["antenna_l"]
Wa = data["antenna_w"]
wavelength = data["wavelength"]
vs = data["orbital_speed"]
h = data["altitude"]
dutycycle = data["dutycycle"]
nadir_duration = data["nadir_duration"]

# Selection
filter by swath width
and
Azimuth resolution

In [5]:
# todo define these as arrays then make a meshgrid for all the combinations
minimum_ground_swath = np.array([20e3])  # m
minimum_azimuth_resolution = np.array([5])  # m

# MESHGRIDS i.e. all combinations
GS, AR = np.meshgrid(minimum_ground_swath, minimum_azimuth_resolution)

# flatten them all
GS = GS.flatten()
AR = AR.flatten()

In [6]:
# unpack the corrected azimuth resolution and swath widths
azimuth_resolutions = []
swathes = []
for dictionary in results_list:
    ar = float(dictionary["corrected-azimuth-resolution-underprocessed"])
    sw = float(dictionary["usable-rasr-swath"])
    azimuth_resolutions.append(ar)
    swathes.append(sw)

In [7]:
# convert dictionaries to numpy arrays
azimuth_resolutions = np.array(azimuth_resolutions)
swathes = np.array(swathes)

In [8]:
# thresholded lists
filtered_designs = []  # list of index vectors

for ii in range(len(AR)):
    indexes1 = np.argwhere(azimuth_resolutions <= AR[ii])
    indexes2 = np.argwhere(swathes >= GS[ii])
    intersection_set = np.intersect1d(indexes1, indexes2)
    filtered_designs.append(intersection_set)

In [9]:
#print(filtered_designs)

# Winning sets visualization on TD diagram
plot a time diagram and display the location of the winning sets

In [16]:
## canonical constraints line plots
# 1 ground range axis in m
ground_range = np.linspace(0, 2000, 500) * 1000

prf_opt = optimal_prf_line(ground_range, Wa, wavelength, h)
# Add to the plot
ax.plot(prf_opt, ground_range / 1000, 'red')

prf_min = minimum_prf_line(ground_range, vs, La, h)
ax.plot(prf_min, ground_range / 1000, 'green')

PRI = 1 / 7050
prf = np.linspace(1 / PRI - 1500, 1 / PRI + 10000, 100)

fig, (ax, ax1) = plt.subplots(2, dpi=150, gridspec_kw={'height_ratios': [30, 1]})
time_diagram_plotter(ax, prf, dutycycle, h, nadir=False, integrationtime=False, color='dimgray')
nadir_return_plotter(ax, prf, dutycycle, nadir_duration, h, color='silver')
ax.set_xlabel('PRF [Hz]')
ax.set_ylabel(' Ground range [km]')
ax.set_xlim(1 / PRI - 1000, 1 / PRI + 1000)
ax.set_ylim(100, 300)
## canonical constraints line plots
ax.plot(prf_opt, ground_range / 1000, 'k')
ax.plot(prf_min, ground_range / 1000, '--k')
ax.set_title('Winners visualization')
fig.tight_layout()

In [17]:
# plotting the winner samples
for ii in filtered_designs[0]:
    ax.plot(x_coordinate[ii], y_coordinate[ii] / 1000, 'x', color='black')
# adjust the plot scale
ax.set_xlim(np.array(x_coordinate).min() - 3e2, np.array(x_coordinate).max() + 3e2)
ax.set_ylim(np.array(y_coordinate).min() / 1e3 - 30, np.array(y_coordinate).max() / 1e3 + 30);

# Boundaries
draw a curve on the T-D plot corresponding to the edges of the swathes within selection

In [18]:
# how do I do that!?